# Data Preparation

## Why do this step?

- The data we scraped from the UFC website is raw data. Every row contains information about a fight that took place i.e. details like how many strikes were thrown in that fight and who won the fight.
- To prepare the data for prediction, every row can contain only an accurate representation of what each fighter has done in fights up until that fight! No data that was recorded during the fight can be present in that row.
- Our Target variable is Winner. The task has to be to predict the winner from the data available of each fighter up until the fight

## Looking at the data

In [1]:
import pandas as pd
import numpy as np
import math

df = pd.read_csv('../data/raw_total_fight_data.csv', sep=';')
fighter_details = pd.read_csv('../data/raw_fighter_details.csv', index_col='fighter_name')

In [2]:
df.head()

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR.,B_SIG_STR.,R_SIG_STR_pct,B_SIG_STR_pct,R_TOTAL_STR.,B_TOTAL_STR.,...,B_GROUND,win_by,last_round,last_round_time,Format,Referee,date,location,Fight_type,Winner
0,Adrian Yanez,Gustavo Lopez,2,0,41 of 103,23 of 51,39%,45%,41 of 103,23 of 51,...,0 of 0,KO/TKO,3,0:27,3 Rnd (5-5-5),Chris Tognoni,"March 20, 2021","Las Vegas, Nevada, USA",Bantamweight Bout,Adrian Yanez
1,Trevin Giles,Roman Dolidze,0,0,27 of 57,32 of 67,47%,47%,43 of 73,75 of 110,...,1 of 2,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Herb Dean,"March 20, 2021","Las Vegas, Nevada, USA",Middleweight Bout,Trevin Giles
2,Tai Tuivasa,Harry Hunsucker,1,0,14 of 18,2 of 6,77%,33%,14 of 18,2 of 6,...,0 of 0,KO/TKO,1,0:49,3 Rnd (5-5-5),Herb Dean,"March 20, 2021","Las Vegas, Nevada, USA",Heavyweight Bout,Tai Tuivasa
3,Cheyanne Buys,Montserrat Conejo,0,0,31 of 65,15 of 41,47%,36%,49 of 87,136 of 168,...,0 of 0,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,"March 20, 2021","Las Vegas, Nevada, USA",Women's Strawweight Bout,Montserrat Conejo
4,Marion Reneau,Macy Chiasson,0,0,30 of 63,51 of 138,47%,36%,59 of 93,92 of 184,...,2 of 4,Decision - Unanimous,3,5:00,3 Rnd (5-5-5),Mark Smith,"March 20, 2021","Las Vegas, Nevada, USA",Women's Bantamweight Bout,Macy Chiasson


In [3]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011
R_fighter,Adrian Yanez,Trevin Giles,Tai Tuivasa,Cheyanne Buys,Marion Reneau,Leonardo Santos,Song Kenan,Derek Brunson,Montel Jackson,Bruno Silva,...,Johnny Rhodes,Royce Gracie,Frank Hamaker,Royce Gracie,Jason DeLucia,Remco Pardoel,Orlando Wiet,Johnny Rhodes,Patrick Smith,Scott Morris
B_fighter,Gustavo Lopez,Roman Dolidze,Harry Hunsucker,Montserrat Conejo,Macy Chiasson,Grant Dawson,Max Griffin,Kevin Holland,Jesse Strader,JP Buys,...,Fred Ettish,Minoki Ichihara,Thaddeus Luster,Jason DeLucia,Scott Baker,Alberta Cerra Leon,Robert Lucarelli,David Levicki,Ray Wizard,Sean Daugherty
R_KD,2,0,1,0,0,0,0,0,2,3,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR.,41 of 103,27 of 57,14 of 18,31 of 65,30 of 63,30 of 67,14 of 24,43 of 93,15 of 26,37 of 61,...,13 of 29,2 of 4,2 of 3,0 of 0,3 of 5,4 of 6,8 of 12,11 of 17,1 of 1,1 of 1
B_SIG_STR.,23 of 51,32 of 67,2 of 6,15 of 41,51 of 138,46 of 84,17 of 28,36 of 74,14 of 25,17 of 56,...,4 of 7,3 of 7,0 of 0,0 of 0,0 of 2,1 of 3,2 of 6,4 of 5,1 of 1,0 of 4
R_SIG_STR_pct,39%,47%,77%,47%,47%,44%,58%,46%,57%,60%,...,44%,50%,66%,---,60%,66%,66%,64%,100%,100%
B_SIG_STR_pct,45%,47%,33%,36%,36%,54%,60%,48%,56%,30%,...,57%,42%,---,---,0%,33%,33%,80%,100%,0%
R_TOTAL_STR.,41 of 103,43 of 73,14 of 18,49 of 87,59 of 93,74 of 115,14 of 24,226 of 323,15 of 26,62 of 89,...,21 of 38,110 of 114,14 of 15,0 of 0,20 of 25,20 of 22,11 of 15,74 of 86,1 of 1,2 of 2
B_TOTAL_STR.,23 of 51,75 of 110,2 of 6,136 of 168,92 of 184,75 of 132,17 of 28,190 of 236,14 of 25,37 of 76,...,7 of 11,12 of 16,0 of 0,0 of 0,14 of 23,9 of 11,2 of 6,95 of 102,2 of 2,1 of 5


In [4]:
df.describe()

,R_KD,B_KD,R_SUB_ATT,B_SUB_ATT,R_REV,B_REV,last_round
count,6012.000000,6012.000000,6012.000000,6012.000000,6012.000000,6012.000000,6012.000000
mean,0.249834,0.179807,0.481371,0.343979,0.137725,0.135396,2.316700
std,0.523408,0.456132,0.924078,0.791808,0.422296,0.414708,1.008284
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000
75%,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3.000000
max,5.000000,4.000000,10.000000,7.000000,5.000000,3.000000,5.000000


In [5]:
df.dtypes

R_fighter          object
B_fighter          object
R_KD                int64
B_KD                int64
R_SIG_STR.         object
B_SIG_STR.         object
R_SIG_STR_pct      object
B_SIG_STR_pct      object
R_TOTAL_STR.       object
B_TOTAL_STR.       object
R_TD               object
B_TD               object
R_TD_pct           object
B_TD_pct           object
R_SUB_ATT           int64
B_SUB_ATT           int64
R_REV               int64
B_REV               int64
R_CTRL             object
B_CTRL             object
R_HEAD             object
B_HEAD             object
R_BODY             object
B_BODY             object
R_LEG              object
B_LEG              object
R_DISTANCE         object
B_DISTANCE         object
R_CLINCH           object
B_CLINCH           object
R_GROUND           object
B_GROUND           object
win_by             object
last_round          int64
last_round_time    object
Format             object
Referee            object
date               object
location    

In [6]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR.', 'B_SIG_STR.',
       'R_SIG_STR_pct', 'B_SIG_STR_pct', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'R_CTRL', 'B_CTRL', 'R_HEAD', 'B_HEAD', 'R_BODY',
       'B_BODY', 'R_LEG', 'B_LEG', 'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH',
       'B_CLINCH', 'R_GROUND', 'B_GROUND', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner'],
      dtype='object')

### Column definitions:

- `R_` and `B_` prefix signifies red and blue corner fighter stats respectively
- `KD` is number of knockdowns
- `SIG_STR` is no. of significant strikes 'landed of attempted'
- `SIG_STR_pct` is significant strikes percentage
- `TOTAL_STR` is total strikes 'landed of attempted'
- `TD` is no. of takedowns
- `TD_pct` is takedown percentages
- `SUB_ATT` is no. of submission attempts
- `PASS` is no. times the guard was passed?
- `REV` is the no. of reversals
- `CTRL` is the time spent with ground control
- `HEAD` is no. of significant strinks to the head 'landed of attempted'
- `BODY` is no. of significant strikes to the body 'landed of attempted'
- `CLINCH` is no. of significant strikes in the clinch 'landed of attempted'
- `GROUND` is no. of significant strikes on the ground 'landed of attempted'
- `win_by` is method of win
- `last_round` is last round of the fight (ex. if it was a KO in 1st, then this will be 1)
- `last_round_time` is when the fight ended in the last round
- `Format` is the format of the fight (3 rounds, 5 rounds etc.)
- `Referee` is the name of the Ref
- `date` is the date of the fight
- `location` is the location in which the event took place
- `Fight_type` is which weight class and whether it's a title bout or not
- `Winner` is the winner of the fight

#### Per fighter career wide stats
- `SLpM` - Significant Strikes Landed per Minute
- `Str_Acc.` - Significant Striking Accuracy
- `SApM` - Significant Strikes Absorbed per Minute
- `Str_Def` - Significant Strike Defence (the % of opponents strikes that did not land)
- `TD_Avg` - Average Takedowns Landed per 15 minutes
- `TD_Acc` - Takedown Accuracy
- `TD_Def` - Takedown Defense (the % of opponents TD attempts that did not land)
- `Sub_Avg` - Average Submissions Attempted per 15 minutes 

## Todo:

- Separate `landed of attempted` to separate columns

- Convert `Fight_type` into two separate columns, `weight_class` and `Title_fight` (True or False)

- Convert `last_round_time` to `total_time_fought` by using `last_round` and `Format`

- Convert `CTRL` to `time_in_CTRL`

- Convert percentages to fractions

- Since the data is a description of each fight, we have to convert it into a format that shows the compilation data of each fighter up until that fight. This means every row will look a lot different than it looks now.

- Create `current_win_streak`, `current_lose_streak`, `longest_win_streak`, `wins`, `losses`, `draw`

- Create fighter `height`, `reach`, `weight`, `age`

In [7]:
fighter_details.drop(
    columns=["SLpM",
            "Str_Acc",
            "SApM",
            "Str_Def",
            "TD_Avg",
            "TD_Acc",
            "TD_Def",
            "Sub_Avg",
        ], inplace=True)

### Splitting landed of attempted to different columns

In [8]:
columns = ['R_SIG_STR.', 'B_SIG_STR.', 'R_TOTAL_STR.', 'B_TOTAL_STR.',
       'R_TD', 'B_TD', 'R_HEAD', 'B_HEAD', 'R_BODY','B_BODY', 'R_LEG', 'B_LEG', 
        'R_DISTANCE', 'B_DISTANCE', 'R_CLINCH','B_CLINCH', 'R_GROUND', 'B_GROUND']

for column in columns:
    print(f"{column} data type is: {df[column].dtype}")

R_SIG_STR. data type is: object
B_SIG_STR. data type is: object
R_TOTAL_STR. data type is: object
B_TOTAL_STR. data type is: object
R_TD data type is: object
B_TD data type is: object
R_HEAD data type is: object
B_HEAD data type is: object
R_BODY data type is: object
B_BODY data type is: object
R_LEG data type is: object
B_LEG data type is: object
R_DISTANCE data type is: object
B_DISTANCE data type is: object
R_CLINCH data type is: object
B_CLINCH data type is: object
R_GROUND data type is: object
B_GROUND data type is: object


In [9]:
attempt_suffix = '_att'
landed_suffix = '_landed'

for column in columns:
    df[column+attempt_suffix] = df[column].apply(lambda X: int(X.split('of')[1]))
    df[column+landed_suffix] = df[column].apply(lambda X: int(X.split('of')[0]))
    
df.drop(columns, axis=1, inplace=True)

In [10]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'R_CTRL', 'B_CTRL', 'win_by', 'last_round',
       'last_round_time', 'Format', 'Referee', 'date', 'location',
       'Fight_type', 'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed'],
      dtype

### Replacing Winner NaNs as Draw

In [11]:
for column in df.columns:
    if df[column].isnull().sum() != 0:
        print(f"NaN values in {column} = {df[column].isnull().sum()}")

NaN values in Referee = 32
NaN values in Winner = 110


* 83 missing values in winner and 23 missing values in Referee

In [12]:
df[df['Winner'].isnull()]['win_by'].value_counts()

Overturned              48
Decision - Majority     25
Could Not Continue      17
Decision - Split        13
Decision - Unanimous     5
Other                    2
Name: win_by, dtype: int64

* Here, Overturned means due to drug test being positive and Could not Continue means there was an illegal blow which was not enough to be disqualified but the fighter could not continue.
* The rest are different forms of draw

* Replacing all of these with draw

In [13]:
df['Winner'].fillna('Draw', inplace=True)

### Converting percentages to fractions

In [14]:
pct_columns = ['R_SIG_STR_pct','B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct']

def pct_to_frac(X):
    if X != '---':
        return float(X.replace('%', ''))/100
    else:
        # if '---' means it's taking pct of `0 of 0`. 
        # Taking a call here to consider 0 landed of 0 attempted as 0 percentage
        return 0

for column in pct_columns:
    df[column] = df[column].apply(pct_to_frac)

In [15]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011
R_fighter,Adrian Yanez,Trevin Giles,Tai Tuivasa,Cheyanne Buys,Marion Reneau,Leonardo Santos,Song Kenan,Derek Brunson,Montel Jackson,Bruno Silva,...,Johnny Rhodes,Royce Gracie,Frank Hamaker,Royce Gracie,Jason DeLucia,Remco Pardoel,Orlando Wiet,Johnny Rhodes,Patrick Smith,Scott Morris
B_fighter,Gustavo Lopez,Roman Dolidze,Harry Hunsucker,Montserrat Conejo,Macy Chiasson,Grant Dawson,Max Griffin,Kevin Holland,Jesse Strader,JP Buys,...,Fred Ettish,Minoki Ichihara,Thaddeus Luster,Jason DeLucia,Scott Baker,Alberta Cerra Leon,Robert Lucarelli,David Levicki,Ray Wizard,Sean Daugherty
R_KD,2,0,1,0,0,0,0,0,2,3,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR_pct,0.39,0.47,0.77,0.47,0.47,0.44,0.58,0.46,0.57,0.6,...,0.44,0.5,0.66,0.0,0.6,0.66,0.66,0.64,1.0,1.0
B_SIG_STR_pct,0.45,0.47,0.33,0.36,0.36,0.54,0.6,0.48,0.56,0.3,...,0.57,0.42,0.0,0.0,0.0,0.33,0.33,0.8,1.0,0.0
R_TD_pct,0.0,0.5,0.0,0.0,0.5,0.5,0.0,0.5,0.0,0.5,...,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
B_TD_pct,0.0,0.33,0.0,0.8,1.0,0.07,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0
R_SUB_ATT,0,1,0,0,0,0,0,1,0,0,...,1,2,3,1,5,1,0,0,1,1
B_SUB_ATT,0,2,0,2,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0


### Creating a title_bout feature and weight_class

In [16]:
df['Fight_type'].value_counts()

Lightweight Bout                                                       1047
Welterweight Bout                                                      1026
Middleweight Bout                                                       769
Heavyweight Bout                                                        530
Featherweight Bout                                                      529
                                                                       ... 
Ultimate Fighter 18 Women's Bantamweight Tournament Title Bout            1
TUF Nations Canada vs. Australia Middleweight Tournament Title Bout       1
TUF Nations Canada vs. Australia Welterweight Tournament Title Bout       1
Ultimate Fighter Brazil 3 Heavyweight Tournament Title Bout               1
UFC 2 Tournament Title Bout                                               1
Name: Fight_type, Length: 109, dtype: int64

In [17]:
df['Fight_type'].value_counts()[df['Fight_type'].value_counts() > 1].index

Index(['Lightweight Bout', 'Welterweight Bout', 'Middleweight Bout',
       'Heavyweight Bout', 'Featherweight Bout', 'Light Heavyweight Bout',
       'Bantamweight Bout', 'Flyweight Bout', 'Women's Strawweight Bout',
       'Women's Bantamweight Bout', 'Women's Flyweight Bout',
       'Open Weight Bout', 'UFC Light Heavyweight Title Bout',
       'UFC Welterweight Title Bout', 'Catch Weight Bout',
       'UFC Heavyweight Title Bout', 'UFC Middleweight Title Bout',
       'UFC Lightweight Title Bout', 'UFC Flyweight Title Bout',
       'UFC Bantamweight Title Bout', 'UFC Featherweight Title Bout',
       'UFC Women's Bantamweight Title Bout',
       'UFC Women's Strawweight Title Bout', 'Women's Featherweight Bout',
       'UFC Women's Featherweight Title Bout',
       'UFC Women's Flyweight Title Bout',
       'UFC Interim Heavyweight Title Bout',
       'UFC Superfight Championship Bout',
       'UFC Interim Welterweight Title Bout',
       'UFC Interim Bantamweight Title Bout',
    

In [18]:
df['title_bout'] = df['Fight_type'].apply(lambda X: True if 'Title Bout' in X else False)

In [19]:
def make_weight_class(X):
    for weight_class in weight_classes:
        if weight_class in X:
            return weight_class
    if X == 'Catch Weight Bout' or 'Catchweight Bout':
        return 'Catch Weight'
    else:
        return 'Open Weight'

In [20]:
weight_classes = ['Women\'s Strawweight', 'Women\'s Bantamweight', 
                  'Women\'s Featherweight', 'Women\'s Flyweight', 'Lightweight', 
                  'Welterweight', 'Middleweight','Light Heavyweight', 
                  'Heavyweight', 'Featherweight','Bantamweight', 'Flyweight', 'Open Weight']

df['weight_class'] = df['Fight_type'].apply(make_weight_class)

In [21]:
df[df['weight_class'].isnull()]['Fight_type'].value_counts()

Series([], Name: Fight_type, dtype: int64)

### Creating total_time_fought

In [22]:
df['Format'].value_counts()

3 Rnd (5-5-5)           5289
5 Rnd (5-5-5-5-5)        507
1 Rnd + OT (12-3)         80
No Time Limit             29
3 Rnd + OT (5-5-5-5)      22
1 Rnd (20)                21
1 Rnd + 2OT (15-3-3)      20
2 Rnd (5-5)               14
1 Rnd (15)                 8
1 Rnd (10)                 6
1 Rnd (12)                 4
1 Rnd + OT (30-5)          3
1 Rnd + OT (15-3)          2
1 Rnd (18)                 2
1 Rnd + OT (27-3)          1
1 Rnd + OT (30-3)          1
1 Rnd (30)                 1
1 Rnd + OT (31-5)          1
1 Rnd + 2OT (24-3-3)       1
Name: Format, dtype: int64

In [23]:
df['Format'].value_counts().index

Index(['3 Rnd (5-5-5)', '5 Rnd (5-5-5-5-5)', '1 Rnd + OT (12-3)',
       'No Time Limit', '3 Rnd + OT (5-5-5-5)', '1 Rnd (20)',
       '1 Rnd + 2OT (15-3-3)', '2 Rnd (5-5)', '1 Rnd (15)', '1 Rnd (10)',
       '1 Rnd (12)', '1 Rnd + OT (30-5)', '1 Rnd + OT (15-3)', '1 Rnd (18)',
       '1 Rnd + OT (27-3)', '1 Rnd + OT (30-3)', '1 Rnd (30)',
       '1 Rnd + OT (31-5)', '1 Rnd + 2OT (24-3-3)'],
      dtype='object')

In [24]:
time_in_first_round = {'3 Rnd (5-5-5)': 5*60, '5 Rnd (5-5-5-5-5)': 5*60, '1 Rnd + OT (12-3)': 12*60,
       'No Time Limit': 1, '3 Rnd + OT (5-5-5-5)': 5*60, '1 Rnd (20)': 1*20,
       '2 Rnd (5-5)': 5*60, '1 Rnd (15)': 15*60, '1 Rnd (10)': 10*60,
       '1 Rnd (12)':12*60, '1 Rnd + OT (30-5)': 30*60, '1 Rnd (18)': 18*60, '1 Rnd + OT (15-3)': 15*60,
       '1 Rnd (30)': 30*60, '1 Rnd + OT (31-5)': 31*5,
       '1 Rnd + OT (27-3)': 27*60, '1 Rnd + OT (30-3)': 30*60}

exception_format_time = {'1 Rnd + 2OT (15-3-3)': [15*60, 3*60], '1 Rnd + 2OT (24-3-3)': [24*60, 3*60]}

# '1 Rnd + 2OT (15-3-3)' and '1 Rnd + 2OT (24-3-3)' is not included because it has 3 uneven timed rounds. 
# We'll have to deal with it separately

In [25]:
# Converting to seconds
df['last_round_time'] = df['last_round_time'].apply(lambda X: int(X.split(':')[0])*60 + int(X.split(':')[1]))

In [26]:
def get_total_time(row):
    if row['Format'] in time_in_first_round.keys():
        return (row['last_round'] - 1) * time_in_first_round[row['Format']] + row['last_round_time']
    elif row['Format'] in exception_format_time.keys():
        if (row['last_round'] - 1) >= 2:
            return exception_format_time[row['Format']][0] + (row['last_round'] - 2) * \
                    exception_format_time[row['Format']][1] + row['last_round_time']
        else:
            return (row['last_round'] - 1) * exception_format_time[row['Format']][0] + row['last_round_time']
    
# So if the fight ended in round 1, we only need last_round_time. 
# If it ended in round 2, we need the full time of round 1 and the last_round_time
# This works for fights with same time in each round and fights with only two rounds.

In [27]:
df['total_time_fought(seconds)'] = df.apply(get_total_time, axis=1)

In [28]:
def get_no_of_rounds(X):
    if X == 'No Time Limit':
        return 1
    else:
        return len(X.split('(')[1].replace(')', '').split('-'))

df['no_of_rounds'] = df['Format'].apply(get_no_of_rounds)

In [29]:
df

,R_fighter,B_fighter,R_KD,B_KD,R_SIG_STR_pct,B_SIG_STR_pct,R_TD_pct,B_TD_pct,R_SUB_ATT,B_SUB_ATT,...,B_CLINCH_att,B_CLINCH_landed,R_GROUND_att,R_GROUND_landed,B_GROUND_att,B_GROUND_landed,title_bout,weight_class,total_time_fought(seconds),no_of_rounds
0,Adrian Yanez,Gustavo Lopez,2,0,0.39,0.45,0.0,0.00,0,0,...,0,0,1,0,0,0,False,Bantamweight,627,3
1,Trevin Giles,Roman Dolidze,0,0,0.47,0.47,0.5,0.33,1,2,...,6,3,10,8,2,1,False,Middleweight,900,3
2,Tai Tuivasa,Harry Hunsucker,1,0,0.77,0.33,0.0,0.00,0,0,...,0,0,8,5,0,0,False,Heavyweight,49,3
3,Cheyanne Buys,Montserrat Conejo,0,0,0.47,0.36,0.0,0.80,0,2,...,0,0,7,3,0,0,False,Women's Strawweight,900,3
4,Marion Reneau,Macy Chiasson,0,0,0.47,0.36,0.5,1.00,0,0,...,15,13,0,0,4,2,False,Women's Bantamweight,900,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,0,0,0.66,0.33,1.0,0.00,1,0,...,0,0,5,4,2,0,False,Open Weight,591,1
6008,Orlando Wiet,Robert Lucarelli,0,0,0.66,0.33,0.0,1.00,0,1,...,0,0,9,7,0,0,False,Open Weight,170,1
6009,Johnny Rhodes,David Levicki,0,0,0.64,0.80,1.0,0.00,0,0,...,2,2,15,9,1,1,False,Open Weight,733,1
6010,Patrick Smith,Ray Wizard,0,0,1.00,1.00,0.0,0.00,1,0,...,0,0,0,0,0,0,False,Open Weight,58,1


In [30]:
df.drop(['Format', 'Fight_type', 'last_round_time'], axis = 1, inplace=True)

### Create CTRL_time(seconds)

In [31]:
CTRL_columns = ['R_CTRL','B_CTRL']

def conv_to_sec(X):
    if X != '--':
        return int(X.split(':')[0])*60 + int(X.split(':')[1])
    else:
        # if '--' means there was no time spent on the ground. 
        # Taking a call here to consider this as 0 seconds
        return 0

for column in CTRL_columns:
    df[column+'_time(seconds)'] = df[column].apply(conv_to_sec)

In [32]:
df.T

,0,1,2,3,4,5,6,7,8,9,...,6002,6003,6004,6005,6006,6007,6008,6009,6010,6011
R_fighter,Adrian Yanez,Trevin Giles,Tai Tuivasa,Cheyanne Buys,Marion Reneau,Leonardo Santos,Song Kenan,Derek Brunson,Montel Jackson,Bruno Silva,...,Johnny Rhodes,Royce Gracie,Frank Hamaker,Royce Gracie,Jason DeLucia,Remco Pardoel,Orlando Wiet,Johnny Rhodes,Patrick Smith,Scott Morris
B_fighter,Gustavo Lopez,Roman Dolidze,Harry Hunsucker,Montserrat Conejo,Macy Chiasson,Grant Dawson,Max Griffin,Kevin Holland,Jesse Strader,JP Buys,...,Fred Ettish,Minoki Ichihara,Thaddeus Luster,Jason DeLucia,Scott Baker,Alberta Cerra Leon,Robert Lucarelli,David Levicki,Ray Wizard,Sean Daugherty
R_KD,2,0,1,0,0,0,0,0,2,3,...,1,0,0,0,0,0,0,0,0,0
B_KD,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
R_SIG_STR_pct,0.39,0.47,0.77,0.47,0.47,0.44,0.58,0.46,0.57,0.6,...,0.44,0.5,0.66,0.0,0.6,0.66,0.66,0.64,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
weight_class,Bantamweight,Middleweight,Heavyweight,Women's Strawweight,Women's Bantamweight,Lightweight,Welterweight,Middleweight,Bantamweight,Flyweight,...,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight,Open Weight
total_time_fought(seconds),627,900,49,900,900,899,140,1500,118,476,...,187,308,292,67,401,591,170,733,58,20
no_of_rounds,3,3,3,3,3,3,3,5,3,3,...,1,1,1,1,1,1,1,1,1,1
R_CTRL_time(seconds),3,75,10,64,135,81,0,1015,16,231,...,0,0,0,0,0,0,0,0,0,0


In [33]:
df.drop(['R_CTRL','B_CTRL'], axis = 1, inplace=True)

### Create another DataFrame to save the compiled data per fighter (Our Prediction DataFrame)

In [34]:
df.columns

Index(['R_fighter', 'B_fighter', 'R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_REV', 'B_REV', 'win_by', 'last_round', 'Referee', 'date', 'location',
       'Winner', 'R_SIG_STR._att', 'R_SIG_STR._landed', 'B_SIG_STR._att',
       'B_SIG_STR._landed', 'R_TOTAL_STR._att', 'R_TOTAL_STR._landed',
       'B_TOTAL_STR._att', 'B_TOTAL_STR._landed', 'R_TD_att', 'R_TD_landed',
       'B_TD_att', 'B_TD_landed', 'R_HEAD_att', 'R_HEAD_landed', 'B_HEAD_att',
       'B_HEAD_landed', 'R_BODY_att', 'R_BODY_landed', 'B_BODY_att',
       'B_BODY_landed', 'R_LEG_att', 'R_LEG_landed', 'B_LEG_att',
       'B_LEG_landed', 'R_DISTANCE_att', 'R_DISTANCE_landed', 'B_DISTANCE_att',
       'B_DISTANCE_landed', 'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att',
       'B_CLINCH_landed', 'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att',
       'B_GROUND_landed', 'title_bout', 'weight_class',
       'total_time_fought(seconds)', 'no_of_rounds',

In [35]:
df2 = df.copy()

In [36]:
df2.drop(['R_KD', 'B_KD', 'R_SIG_STR_pct',
       'B_SIG_STR_pct', 'R_TD_pct', 'B_TD_pct', 'R_SUB_ATT', 'B_SUB_ATT',
       'R_CTRL_time(seconds)', 'B_CTRL_time(seconds)', 'R_REV', 'B_REV', 'win_by', 'last_round', 
        'R_SIG_STR._att', 'R_SIG_STR._landed',
       'B_SIG_STR._att', 'B_SIG_STR._landed', 'R_TOTAL_STR._att',
       'R_TOTAL_STR._landed', 'B_TOTAL_STR._att', 'B_TOTAL_STR._landed',
       'R_TD_att', 'R_TD_landed', 'B_TD_att', 'B_TD_landed', 'R_HEAD_att',
       'R_HEAD_landed', 'B_HEAD_att', 'B_HEAD_landed', 'R_BODY_att',
       'R_BODY_landed', 'B_BODY_att', 'B_BODY_landed', 'R_LEG_att',
       'R_LEG_landed', 'B_LEG_att', 'B_LEG_landed', 'R_DISTANCE_att',
       'R_DISTANCE_landed', 'B_DISTANCE_att', 'B_DISTANCE_landed',
       'R_CLINCH_att', 'R_CLINCH_landed', 'B_CLINCH_att', 'B_CLINCH_landed',
       'R_GROUND_att', 'R_GROUND_landed', 'B_GROUND_att', 'B_GROUND_landed',
        'total_time_fought(seconds)'], axis = 1, inplace=True)
df2

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,"March 20, 2021","Las Vegas, Nevada, USA",Adrian Yanez,False,Bantamweight,3
1,Trevin Giles,Roman Dolidze,Herb Dean,"March 20, 2021","Las Vegas, Nevada, USA",Trevin Giles,False,Middleweight,3
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,"March 20, 2021","Las Vegas, Nevada, USA",Tai Tuivasa,False,Heavyweight,3
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,"March 20, 2021","Las Vegas, Nevada, USA",Montserrat Conejo,False,Women's Strawweight,3
4,Marion Reneau,Macy Chiasson,Mark Smith,"March 20, 2021","Las Vegas, Nevada, USA",Macy Chiasson,False,Women's Bantamweight,3
...,...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Remco Pardoel,False,Open Weight,1
6008,Orlando Wiet,Robert Lucarelli,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Orlando Wiet,False,Open Weight,1
6009,Johnny Rhodes,David Levicki,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Johnny Rhodes,False,Open Weight,1
6010,Patrick Smith,Ray Wizard,John McCarthy,"March 11, 1994","Denver, Colorado, USA",Patrick Smith,False,Open Weight,1


### Compiling Data per fighter

In [37]:
red_fighters = df['R_fighter'].value_counts().index
blue_fighters = df['B_fighter'].value_counts().index

fighters = list(set(red_fighters) | set(blue_fighters))

In [38]:
def get_renamed_winner(row):
    if row['R_fighter'] == row['Winner']:
        return 'Red'
    elif row['B_fighter'] == row['Winner']:
        return 'Blue'
    elif row['Winner'] == 'Draw':
        return 'Draw'

df2['Winner'] = df2[['R_fighter', 'B_fighter', 'Winner']].apply(get_renamed_winner, axis=1)

In [39]:
df = pd.concat([df,pd.get_dummies(df['win_by'], prefix='win_by')],axis=1)
df.drop(['win_by'],axis=1, inplace=True)

In [40]:
Numerical_columns = ['hero_KD', 'opp_KD', 'hero_SIG_STR_pct',
       'opp_SIG_STR_pct', 'hero_TD_pct', 'opp_TD_pct', 'hero_SUB_ATT', 'opp_SUB_ATT',
        'hero_REV', 'opp_REV', 'hero_SIG_STR._att', 'hero_SIG_STR._landed',
       'opp_SIG_STR._att', 'opp_SIG_STR._landed', 'hero_TOTAL_STR._att',
       'hero_TOTAL_STR._landed', 'opp_TOTAL_STR._att', 'opp_TOTAL_STR._landed',
       'hero_TD_att', 'hero_TD_landed', 'opp_TD_att', 'opp_TD_landed', 'hero_HEAD_att',
       'hero_HEAD_landed', 'opp_HEAD_att', 'opp_HEAD_landed', 'hero_BODY_att',
       'hero_BODY_landed', 'opp_BODY_att', 'opp_BODY_landed', 'hero_LEG_att',
       'hero_LEG_landed', 'opp_LEG_att', 'opp_LEG_landed', 'hero_DISTANCE_att',
       'hero_DISTANCE_landed', 'opp_DISTANCE_att', 'opp_DISTANCE_landed',
       'hero_CLINCH_att', 'hero_CLINCH_landed', 'opp_CLINCH_att', 'opp_CLINCH_landed',
       'hero_GROUND_att', 'hero_GROUND_landed', 'opp_GROUND_att', 'opp_GROUND_landed', 
        'hero_CTRL_time(seconds)', 'opp_CTRL_time(seconds)',
       'total_time_fought(seconds)']

Categorical_columns = ['win_by', 'last_round',
        'Winner', 'title_bout']

For all `Numerical_columns`, we take the average of those columns for every fighter of every fight they had up until that point.

For `Categorical_columns`, we have to come up with different ideas for each column:

* Each `win_by` will be a column of it's own
* from `last_round` we can get, `total_rounds_fought`
* from `total_time_fought` we can get `average_time_fought`
* from `Winner` we get `wins`, `losses`, `draw`, `current_streak`, `longest_streak`
* from `title_bout` we can get `no_of_title_fights`

In [41]:
import re

def lreplace(pattern, sub, string):
    """
    Replaces 'pattern' in 'string' with 'sub' if 'pattern' starts 'string'.
    """
    return re.sub('^%s' % pattern, sub, string)

In [42]:
red = df.groupby('R_fighter')
blue = df.groupby('B_fighter')

In [43]:
def get_fighter_red(fighter_name):
    try:
        fighter_red = red.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_red.columns:
        if re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'hero_', column)
        elif re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'opp_', column)
    fighter_red = fighter_red.rename(rename_columns, axis='columns')
    return fighter_red

In [44]:
def get_fighter_blue(fighter_name):
    try:
        fighter_blue = blue.get_group(fighter_name)
    except:
        return None
    rename_columns = {}
    for column in fighter_blue.columns:
        if re.search('^B_', column) is not None:
            rename_columns[column] = lreplace('B_', 'hero_', column)
        elif re.search('^R_', column) is not None:
            rename_columns[column] = lreplace('R_', 'opp_', column)
    fighter_blue = fighter_blue.rename(rename_columns, axis='columns')
    return fighter_blue

In [45]:
def get_result_stats(result_list):
    result_list.reverse() # To get it in ascending order
    current_win_streak = 0
    current_lose_streak = 0
    longest_win_streak = 0
    wins = 0
    losses = 0
    draw = 0
    for result in result_list:
        if result == 'hero':
            wins += 1
            current_win_streak += 1
            current_lose_streak = 0
            if longest_win_streak < current_win_streak:
                longest_win_streak += 1
        elif result == 'opp':
            losses += 1
            current_win_streak = 0
            current_lose_streak += 1
        elif result == 'draw':
            draw += 1
            current_lose_streak = 0
            current_win_streak = 0
            
    return current_win_streak, current_lose_streak, longest_win_streak, wins, losses, draw

In [46]:
win_by_columns = ['win_by_Decision - Majority', 'win_by_Decision - Split',
       'win_by_Decision - Unanimous', 'win_by_KO/TKO','win_by_Submission',
       'win_by_TKO - Doctor\'s Stoppage']

In [47]:
temp_blue_frame = pd.DataFrame()
temp_red_frame = pd.DataFrame()
result_stats = ['current_win_streak', 'current_lose_streak', 'longest_win_streak', 'wins', 'losses', 'draw']

for fighter_name in fighters:
    fighter_red = get_fighter_red(fighter_name)
    fighter_blue = get_fighter_blue(fighter_name)
    fighter_index = None
    
    if fighter_red is None:
        fighter = fighter_blue
        fighter_index = 'blue'
    elif fighter_blue is None:
        fighter = fighter_red
        fighter_index = 'red'
    else:
        fighter = pd.concat([fighter_red, fighter_blue]).sort_index()
    
    fighter['Winner'] = fighter['Winner'].apply(lambda X: 'hero' if X == fighter_name else 'opp')

    for i, index in enumerate(fighter.index):
        fighter_slice = fighter[(i+1):].sort_index(ascending=False)
        s = fighter_slice[Numerical_columns].ewm(span=3, adjust=False).mean().tail(1)
        if len(s) != 0:
            pass
        else:
            s.loc[len(s)] = [np.NaN for _ in s.columns]
        s['total_rounds_fought'] = fighter_slice['last_round'].sum()
        s['total_title_bouts'] = fighter_slice[fighter_slice['title_bout']==True]['title_bout'].count()
        s['hero_fighter'] = fighter_name
        results = get_result_stats(list(fighter_slice['Winner']))
        for result_stat, result in zip(result_stats, results):
            s[result_stat] = result
        win_by_results = fighter_slice[fighter_slice['Winner'] == 'hero'][win_by_columns].sum()
        for win_by_column,win_by_result in zip(win_by_columns, win_by_results):
            s[win_by_column] = win_by_result
        s.index = [index]


        if fighter_index is None:
            if index in fighter_blue.index:
                temp_blue_frame = temp_blue_frame.append(s)
            elif index in fighter_red.index:
                temp_red_frame = temp_red_frame.append(s)
        elif fighter_index == 'blue':
            temp_blue_frame = temp_blue_frame.append(s)
        elif fighter_index == 'red':
            temp_red_frame = temp_red_frame.append(s)

In [48]:
temp_blue_frame.T

,1845,2232,2378,2699,2038,2219,3115,4369,4434,4528,...,1056,1509,1703,2338,2567,4355,4546,635,1528,2175
hero_KD,0.375,0.75,0.5,NaN,0.032227,0.064453,0.515625,0.0,0.0,NaN,...,0.25,0.5,0.0,0.0,NaN,0.0,NaN,0.09375,0.75,NaN
opp_KD,0.125,0.25,0.5,NaN,0.5,0.0,0.0,0.0,0.0,NaN,...,0.1875,0.375,0.75,1.0,NaN,1.0,NaN,0.0,0.0,NaN
hero_SIG_STR_pct,0.48625,0.6225,0.795,NaN,0.397942,0.355884,0.32707,0.46,0.53,NaN,...,0.513125,0.59625,0.5425,0.45,NaN,0.46,NaN,0.612812,0.4525,NaN
opp_SIG_STR_pct,0.43625,0.4925,0.455,NaN,0.46613,0.492261,0.378086,0.46,0.27,NaN,...,0.49625,0.6225,0.585,0.59,NaN,0.42,NaN,0.23,0.47,NaN
hero_TD_pct,0.23,0.46,0.5,NaN,0.01208,0.02416,0.193281,1.0,0.0,NaN,...,0.16625,0.3325,0.665,0.0,NaN,1.0,NaN,0.574375,0.395,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
win_by_Decision - Split,0.0,0.0,0.0,0.0,3.0,3.0,3.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
win_by_Decision - Unanimous,0.0,0.0,0.0,0.0,4.0,4.0,3.0,0.0,1.0,0.0,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0
win_by_KO/TKO,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0
win_by_Submission,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0


### Adding fighter details like height, weight, reach, stance and dob

In [49]:
fighter_details.columns

Index(['Height', 'Weight', 'Reach', 'Stance', 'DOB'], dtype='object')

In [50]:
fighter_details = fighter_details[fighter_details.index.isin(fighters)]
for col in fighter_details.columns:
    print(f"Number of NaN in {col} : {fighter_details[col].isnull().sum()}")

Number of NaN in Height : 13
Number of NaN in Weight : 10
Number of NaN in Reach : 649
Number of NaN in Stance : 75
Number of NaN in DOB : 141


In [51]:
fighter_details.columns

Index(['Height', 'Weight', 'Reach', 'Stance', 'DOB'], dtype='object')

In [52]:
def convert_to_cms(X):
    if X is np.NaN:
        return X
    elif len(X.split("'")) == 2:
        feet = float(X.split("'")[0])
        inches = int(X.split("'")[1].replace(' ', '').replace('"',''))
        return (feet * 30.48) + (inches * 2.54)
    else:
        return float(X.replace('"','')) * 2.54

In [53]:
fighter_details['Height_cms'] = fighter_details['Height'].apply(convert_to_cms)
fighter_details['Reach_cms'] = fighter_details['Reach'].apply(convert_to_cms)

In [54]:
fighter_details.columns

Index(['Height', 'Weight', 'Reach', 'Stance', 'DOB', 'Height_cms',
       'Reach_cms'],
      dtype='object')

In [55]:
fighter_details['Weight_lbs'] = fighter_details['Weight'].apply(lambda X: float(X.replace(' lbs.', '')) if X is not np.NaN else X)

In [56]:
fighter_details.columns

Index(['Height', 'Weight', 'Reach', 'Stance', 'DOB', 'Height_cms', 'Reach_cms',
       'Weight_lbs'],
      dtype='object')

In [58]:
pct_columns = ['Str_Acc','Str_Def', 'TD_Acc', 'TD_Def']

def pct_to_frac(X):
    if X != np.NaN:
        return float(X.replace('%', ''))/100
    else:
        return 0

In [60]:
fighter_details.columns

Index(['Height', 'Weight', 'Reach', 'Stance', 'DOB', 'Height_cms', 'Reach_cms',
       'Weight_lbs'],
      dtype='object')

In [61]:
fighter_details.drop(['Height', 'Weight', 'Reach'], axis=1, inplace=True)

In [62]:
fighter_details.reset_index(inplace=True)
temp_red_frame.reset_index(inplace=True)
temp_blue_frame.reset_index(inplace=True)

In [63]:
temp_blue_frame = temp_blue_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_blue_frame.set_index('index', inplace=True)

In [64]:
temp_blue_frame[['hero_fighter', 'fighter_name', 'Height_cms', 'Weight_lbs', 'DOB']].head(20)

,hero_fighter,fighter_name,Height_cms,Weight_lbs,DOB
index,,,,,
1845,Dominique Steele,Dominique Steele,177.80,170.0,"Jan 27, 1988"
2232,Dominique Steele,Dominique Steele,177.80,170.0,"Jan 27, 1988"
2378,Dominique Steele,Dominique Steele,177.80,170.0,"Jan 27, 1988"
2699,Dominique Steele,Dominique Steele,177.80,170.0,"Jan 27, 1988"
2038,Takeya Mizugaki,Takeya Mizugaki,170.18,135.0,"Dec 16, 1983"
2219,Takeya Mizugaki,Takeya Mizugaki,170.18,135.0,"Dec 16, 1983"
3115,Takeya Mizugaki,Takeya Mizugaki,170.18,135.0,"Dec 16, 1983"
4369,Takeya Mizugaki,Takeya Mizugaki,170.18,135.0,"Dec 16, 1983"
4434,Kyle Watson,Kyle Watson,182.88,155.0,"Aug 09, 1979"


In [65]:
temp_red_frame = temp_red_frame.merge(fighter_details, left_on='hero_fighter', right_on='fighter_name', how='left')
temp_red_frame.set_index('index', inplace=True)

In [66]:
temp_blue_frame.drop('fighter_name', axis=1, inplace=True)
temp_red_frame.drop('fighter_name', axis=1, inplace=True)

In [67]:
blue_frame = temp_blue_frame.add_prefix('B_')
red_frame = temp_red_frame.add_prefix('R_')

In [68]:
frame = blue_frame.join(red_frame, how='outer')

In [69]:
rename_cols = {}
for col in frame.columns:
    if 'hero' in col:
        rename_cols[col] = col.replace('_hero_', '_avg_').replace('.', '')
    if 'opp' in col:
        rename_cols[col] = col.replace('_opp_', '_avg_opp_').replace('.', '')
    if 'win_by' in col:
        rename_cols[col] = col.replace(' ', '').replace('-', '_').replace('\'s', '_')

In [70]:
frame.rename(rename_cols, axis='columns', inplace=True)

In [71]:
frame.drop(['R_avg_fighter','B_avg_fighter'], axis=1, inplace=True)

In [72]:
df2 = df2.join(frame, how='outer')

### Create Age

In [73]:
df2['R_DOB'] = pd.to_datetime(df2['R_DOB'])
df2['B_DOB'] = pd.to_datetime(df2['B_DOB'])
df2['date'] = pd.to_datetime(df2['date'])

In [74]:
def get_age(row):
    B_age = (row['date'] - row['B_DOB']).days
    R_age = (row['date'] - row['R_DOB']).days
    if np.isnan(B_age)!=True:
        B_age = math.floor(B_age/365.25)
    if np.isnan(R_age)!=True:
        R_age = math.floor(R_age/365.25)
    return pd.Series([B_age, R_age], index=['B_age', 'R_age'])

In [75]:
df2[['B_age', 'R_age']]= df2[['date', 'R_DOB', 'B_DOB']].apply(get_age, axis=1)

In [76]:
df2.drop(['R_DOB', 'B_DOB'], axis=1, inplace=True)

In [78]:
df2.head()

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_avg_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,3,0.000,...,0.0,1.0,0.0,0.0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,3,0.500,...,0.0,3.0,0.0,0.0,Orthodox,182.88,187.96,185.0,32.0,28.0
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,3,NaN,...,1.0,3.0,0.0,0.0,Southpaw,187.96,190.50,264.0,32.0,28.0
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,Women's Strawweight,3,NaN,...,0.0,0.0,0.0,0.0,Switch,160.02,160.02,115.0,28.0,25.0
4,Marion Reneau,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,Women's Bantamweight,3,0.125,...,1.0,2.0,2.0,0.0,Orthodox,167.64,172.72,135.0,29.0,43.0


In [79]:
df2.to_csv('../data/data.csv', index=False)

In [80]:
df2

,R_fighter,B_fighter,Referee,date,location,Winner,title_bout,weight_class,no_of_rounds,B_avg_KD,...,R_win_by_Decision_Unanimous,R_win_by_KO/TKO,R_win_by_Submission,R_win_by_TKO_Doctor_Stoppage,R_Stance,R_Height_cms,R_Reach_cms,R_Weight_lbs,B_age,R_age
0,Adrian Yanez,Gustavo Lopez,Chris Tognoni,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Bantamweight,3,0.000,...,0.0,1.0,0.0,0.0,Orthodox,170.18,177.80,135.0,31.0,27.0
1,Trevin Giles,Roman Dolidze,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Middleweight,3,0.500,...,0.0,3.0,0.0,0.0,Orthodox,182.88,187.96,185.0,32.0,28.0
2,Tai Tuivasa,Harry Hunsucker,Herb Dean,2021-03-20,"Las Vegas, Nevada, USA",Red,False,Heavyweight,3,NaN,...,1.0,3.0,0.0,0.0,Southpaw,187.96,190.50,264.0,32.0,28.0
3,Cheyanne Buys,Montserrat Conejo,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,Women's Strawweight,3,NaN,...,0.0,0.0,0.0,0.0,Switch,160.02,160.02,115.0,28.0,25.0
4,Marion Reneau,Macy Chiasson,Mark Smith,2021-03-20,"Las Vegas, Nevada, USA",Blue,False,Women's Bantamweight,3,0.125,...,1.0,2.0,2.0,0.0,Orthodox,167.64,172.72,135.0,29.0,43.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6007,Remco Pardoel,Alberta Cerra Leon,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.0,0.0,0.0,0.0,Southpaw,190.50,NaN,260.0,NaN,NaN
6008,Orlando Wiet,Robert Lucarelli,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.0,0.0,0.0,0.0,Southpaw,177.80,NaN,170.0,NaN,NaN
6009,Johnny Rhodes,David Levicki,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.0,0.0,0.0,0.0,Orthodox,182.88,NaN,210.0,NaN,NaN
6010,Patrick Smith,Ray Wizard,John McCarthy,1994-03-11,"Denver, Colorado, USA",Red,False,Open Weight,1,NaN,...,0.0,0.0,0.0,0.0,Orthodox,187.96,NaN,225.0,NaN,30.0
